In [18]:
import pandas as pd

In [45]:
!pip install openpyx1

ERROR: Could not find a version that satisfies the requirement openpyx1 (from versions: none)
ERROR: No matching distribution found for openpyx1


In [43]:
df = pd.read_excel('../data/raw_data/mapping_columns.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [41]:
df

,old_name\tnew_name
0,MPA RAG rating\tcolour_rating
1,Description / Aims \tdescription_aims
2,Description / Aims\tdescription_aims
3,Departmental commentary on actions planned or ...
4,Project - Start Date\tstart_date
5,Departmental commentary on actions planned or ...
6,Project - End Date\tend_date
7,Project - Start Date \tstart_date
8,"Departmental narrative on schedule, including ..."
9,2013/2014 Budget (£million)\tyearly_budget


In [24]:
df1 = pd.read_csv('../data/raw_data/col_names.csv')

In [25]:
df1

,Unnamed: 0,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,0,project_name,project_name,project_name,project_name,project_name,project_name,project_name,project_name,project_name,project_name
1,1,Department,Department,Department,Department,Department,Department,Department,Department,Department,Department
2,2,MPA RAG rating\n(A Delivery Confidence Assessm...,MPA RAG rating\n(A Delivery Confidence Assessm...,IPA RAG rating\n(A Delivery Confidence Assessm...,IPA Delivery Confidence Assessment\n(A Deliver...,IPA Delivery Confidence Assessment\r(A Deliver...,IPA Delivery Confidence Assessment\n(A Deliver...,Description / Aims,Annual Report Category,Annual Report Category,Annual Report Category
3,3,Description / Aims \n,Description / Aims \n(From GMPP data),Description / Aims,Description / Aims,Description / Aims,Description / Aims,IPA Delivery Confidence Assessment\n(A Deliver...,Description / Aims,Description / Aims,Description / Aims
4,4,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,IPA Delivery Confidence Assessment \n(A Delive...,IPA Delivery Confidence Assessment \n(A Delive...,IPA Delivery Confidence Assessment \n(A Delive...
5,5,Project - Start Date\n(Latest approved start d...,Project - Start Date\n(Latest approved start d...,Project - Start Date\n(Latest approved start d...,Project - Start Date\n(Latest approved start d...,Project - Start Date,NaN,NaN,NaN,NaN,NaN
6,(Latest approved start date),Project - Start Date\n(Latest Approved Start D...,Project - Start Date\n(Latest Approved Start D...,Departmental commentary on actions planned or ...,SRO Delivery Confidence Assessment \n(A Delive...,SRO Delivery Confidence Assessment \n(A Delive...,NaN,NaN,NaN,NaN,NaN
7,6,Project - End Date\n(Latest approved end date),Project - End Date\n(Latest approved end date),Project - End Date\n(Latest approved end date),Project - End Date\n(Latest approved end date),Project - End Date,NaN,NaN,NaN,NaN,NaN
8,(Latest approved end date),Project - End Date\n(Latest Approved End Date),Project - End Date\n(Latest Approved End Date),Project - Start Date \n(Latest Approved Start ...,Departmental commentary on actions planned or ...,Departmental commentary on actions planned or ...,NaN,NaN,NaN,NaN,NaN
9,7,"Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...","Departmental narrative on schedule, including ...",Project - End Date \n(Latest Approved End Date),Project - Start Date \n(Latest Approved Start ...,Project - Start Date \n(Latest Approved Start ...
